## Boundless DAS

In [1]:
__author__ = "Zhengxuan Wu"
__version__ = "10/05/2023"

### Overview

This tutorial aims to reproduce one key result of [the Boundless DAS paper](https://arxiv.org/pdf/2305.08809). It uses the same pricing tag dataset as in the paper. Additionally, it focuses on finding alignment for the left boundary check only. 

### Set-up

In [1]:
# This library is our indicator that the required installs
# need to be done.
import pyvene



In [2]:
import torch
from tqdm import tqdm, trange
from datasets import Dataset
from torch.utils.data import DataLoader
from transformers import get_linear_schedule_with_warmup
from torch.nn import CrossEntropyLoss
from tutorial_price_tagging_utils import (
    factual_sampler,
    bound_alignment_sampler,
    lower_bound_alignment_example_sampler,
)

from pyvene import (
    IntervenableModel,
    BoundlessRotatedSpaceIntervention,
    RepresentationConfig,
    IntervenableConfig,
)
from pyvene import create_llama
from pyvene import set_seed, count_parameters

In [3]:
config, tokenizer, llama = create_llama()
_ = llama.to("cuda")  # single gpu
_ = llama.eval()  # always no grad on the model

`torch_dtype` is deprecated! Use `dtype` instead!


Loading checkpoint shards:   0%|          | 0/34 [00:00<?, ?it/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you loaded a llama tokenizer from a GGUF file you can ignore this message


loaded model


### Factual performance of instruct-tuned LLaMA-7B

In [4]:
raw_prealign = factual_sampler(tokenizer, 5000, game="pricing_tag")
prealign_dataset = Dataset.from_dict(
    {"input_ids": raw_prealign[0], "labels": raw_prealign[1]}
)
prealign_dataset.set_format("torch", columns=["input_ids", "labels"])
prealign_dataloader = DataLoader(prealign_dataset, batch_size=8)

In [5]:
total_count = 0
correct_count = 0
with torch.no_grad():
    for step, inputs in enumerate(tqdm(prealign_dataloader)):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to(llama.device)

        # aligning forward!
        outputs = llama(
            input_ids=inputs["input_ids"],
            labels=inputs["labels"],
        )

        actual_test_labels = inputs["labels"][:, -1]
        pred_test_labels = torch.argmax(outputs.logits[:, -1], dim=-1)

        correct_labels = actual_test_labels == pred_test_labels

        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
current_acc = round(correct_count / total_count, 2)
print(f"[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: {current_acc}")

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 625/625 [01:42<00:00,  6.12it/s]

[WARNING: THIS NEEDS TO BE GOOD!] prealign task accuracy: 0.92


### Create training dataset for our trainable intervention (Boundless DAS)

In [6]:
set_seed(42)

###################
# data loaders
###################
raw_data = bound_alignment_sampler(
    tokenizer, 10000, [lower_bound_alignment_example_sampler]
)

raw_train = (
    raw_data[0][:8000],
    raw_data[1][:8000],
    raw_data[2][:8000],
    raw_data[3][:8000],
)
raw_eval = (
    raw_data[0][8000:9000],
    raw_data[1][8000:9000],
    raw_data[2][8000:9000],
    raw_data[3][8000:9000],
)
raw_test = (
    raw_data[0][9000:],
    raw_data[1][9000:],
    raw_data[2][9000:],
    raw_data[3][9000:],
)
train_dataset = Dataset.from_dict(
    {
        "input_ids": raw_train[0],
        "source_input_ids": raw_train[1],
        "labels": raw_train[2],
        "intervention_ids": raw_train[3],  # we will not use this field
    }
).with_format("torch")
train_dataloader = DataLoader(
    train_dataset,
    batch_size=16,
)
eval_dataset = Dataset.from_dict(
    {
        "input_ids": raw_eval[0],
        "source_input_ids": raw_eval[1],
        "labels": raw_eval[2],
        "intervention_ids": raw_eval[3],  # we will not use this field
    }
).with_format("torch")
eval_dataloader = DataLoader(
    eval_dataset,
    batch_size=16,
)
test_dataset = Dataset.from_dict(
    {
        "input_ids": raw_test[0],
        "source_input_ids": raw_test[1],
        "labels": raw_test[2],
        "intervention_ids": raw_test[3],  # we will not use this field
    }
).with_format("torch")
test_dataloader = DataLoader(
    test_dataset,
    batch_size=16,
)

### Boundless DAS on Position-aligned Tokens

In [7]:
def simple_boundless_das_position_config(model_type, intervention_type, layer):
    config = IntervenableConfig(
        model_type=model_type,
        representations=[
            RepresentationConfig(
                layer,              # layer
                intervention_type,  # intervention type
            ),
        ],
        intervention_types=BoundlessRotatedSpaceIntervention,
    )
    return config


config = simple_boundless_das_position_config(
    type(llama), "block_output", 15
)
intervenable = IntervenableModel(config, llama)
intervenable.set_device("cuda")
intervenable.disable_model_gradients()

In [8]:
t_total = int(len(train_dataloader) * 3)
warm_up_steps = 0.1 * t_total
optimizer_params = []
for k, v in intervenable.interventions.items():
    optimizer_params += [{"params": v[0].rotate_layer.parameters()}]
    optimizer_params += [{"params": v[0].intervention_boundaries, "lr": 1e-2}]
optimizer = torch.optim.Adam(optimizer_params, lr=1e-3)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=warm_up_steps, num_training_steps=t_total
)


# You can define your custom compute_metrics function.
def compute_metrics(eval_preds, eval_labels):
    total_count = 0
    correct_count = 0
    for eval_pred, eval_label in zip(eval_preds, eval_labels):
        actual_test_labels = eval_label[:, -1]
        pred_test_labels = torch.argmax(eval_pred[:, -1], dim=-1)
        correct_labels = actual_test_labels == pred_test_labels
        total_count += len(correct_labels)
        correct_count += correct_labels.sum().tolist()
    accuracy = round(correct_count / total_count, 2)
    return {"accuracy": accuracy}


epochs = 3
gradient_accumulation_steps = 4
total_step = 0
target_total_step = len(train_dataloader) * epochs
temperature_start = 50.0
temperature_end = 0.1
temperature_schedule = (
    torch.linspace(temperature_start, temperature_end, target_total_step)
    .to(torch.bfloat16)
    .to("cuda")
)
intervenable.set_temperature(temperature_schedule[total_step])


def calculate_loss(logits, labels):
    shift_logits = logits[..., :, :].contiguous()
    shift_labels = labels[..., :].contiguous()
    # Flatten the tokens
    loss_fct = CrossEntropyLoss()
    shift_logits = shift_logits.view(-1, intervenable.model_config.vocab_size)
    shift_labels = shift_labels.view(-1)
    # Enable model parallelism
    shift_labels = shift_labels.to(shift_logits.device)
    loss = loss_fct(shift_logits, shift_labels)

    for k, v in intervenable.interventions.items():
        boundary_loss = 1.0 * v[0].intervention_boundaries.sum()
    loss += boundary_loss

    return loss

In [9]:
intervenable.model.train()  # train enables drop-off but no grads
print("llama trainable parameters: ", count_parameters(intervenable.model))
print("intervention trainable parameters: ", intervenable.count_parameters())
train_iterator = trange(0, int(epochs), desc="Epoch")
for epoch in train_iterator:
    epoch_iterator = tqdm(
        train_dataloader, desc=f"Epoch: {epoch}", position=0, leave=True
    )
    for step, inputs in enumerate(epoch_iterator):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to("cuda")
        b_s = inputs["input_ids"].shape[0]
        _, counterfactual_outputs = intervenable(
            {"input_ids": inputs["input_ids"]},
            [{"input_ids": inputs["source_input_ids"]}],
            {"sources->base": 80},  # swap 80th token
        )
        eval_metrics = compute_metrics(
            [counterfactual_outputs.logits], [inputs["labels"]]
        )

        # loss and backprop
        loss = calculate_loss(counterfactual_outputs.logits, inputs["labels"])
        loss_str = round(loss.item(), 2)
        epoch_iterator.set_postfix({"loss": loss_str, "acc": eval_metrics["accuracy"]})

        if gradient_accumulation_steps > 1:
            loss = loss / gradient_accumulation_steps
        loss.backward()
        if total_step % gradient_accumulation_steps == 0:
            if not (gradient_accumulation_steps > 1 and total_step == 0):
                optimizer.step()
                scheduler.step()
                intervenable.set_zero_grad()
                intervenable.set_temperature(temperature_schedule[total_step])
        total_step += 1

llama trainable parameters:  0
intervention trainable parameters:  16777218


Epoch:   0%|                                                                                                                                                    | 0/3 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [10]:
# evaluation on the test set
eval_labels = []
eval_preds = []
base_hstates = []
intrv_hstates = []
with torch.no_grad():
    epoch_iterator = tqdm(test_dataloader, desc=f"Test")
    for step, inputs in enumerate(epoch_iterator):
        for k, v in inputs.items():
            if v is not None and isinstance(v, torch.Tensor):
                inputs[k] = v.to("cuda")
        b_s = inputs["input_ids"].shape[0]
        base_outputs, counterfactual_outputs = intervenable(
            {"input_ids": inputs["input_ids"]},
            [{"input_ids": inputs["source_input_ids"]}],
            {"sources->base": 80},  # swap 80th token
            output_original_output=True,
            output_hidden_states=True,
        )
        eval_labels += [inputs["labels"]]
        eval_preds += [counterfactual_outputs.logits]
        base_hstates.append([h.cpu() for h in base_outputs.hidden_states])
        intrv_hstates.append([h.cpu() for h in counterfactual_outputs.hidden_states])
eval_metrics = compute_metrics(eval_preds, eval_labels)
print(eval_metrics)

Test:  10%|█████████████▎                                                                                                                              | 6/63 [00:09<01:32,  1.62s/it]


KeyboardInterrupt: 

In [1]:

for layer in range(len(intrv_hstates[0])):
    do_break = False
    for pos in range(intrv_hstates[0][layer].shape[1]):
        if not torch.all( intrv_hstates[0][layer][:,pos] == base_hstates[0][layer][:,pos] ):
            d = base_hstates[0][layer].shape[-1]
            natty_states = [base_hstates[i][layer][:,pos].reshape(-1,d) for i in range(len(base_hstates))]
            intrv_states = [intrv_hstates[i][layer][:,pos].reshape(-1,d) for i in range(len(base_hstates))]
            print("L:", layer, "Pos:", pos)
            do_break = True
            break
    if do_break: break
natty_states = torch.vstack(natty_states)
intrv_states = torch.vstack(intrv_states)
print(intrv_states.shape)

NameError: name 'intrv_hstates' is not defined

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

def interleave_scatter(
    natty_df,
    intrv_df,
    hue="count",
    title=None,
    samps_per_step=50,
    incl_legend=False,
    leg_loc=None,
    leg_bbox_anchor=None,
    leg_alpha=0.7,
    fontsize=25,
    titlesize=30,
    legendsize=25,
    ticksize=30,
    labelsize=25,
    save_name=None,
    *args, **kwargs,
):
    #df = pd.DataFrame({
    #    "x": samples[:,0],
    #    "y": samples[:,1],
    #    "hue": samples[:,1],
    #})
    #df["x"] = (df["x"]-np.mean(df["x"]))
    #df["hue"] = df["hue"]-np.min(df["hue"])
    #df["hue"] = df["hue"]/np.max(df["hue"])
    
    
    
    fig = plt.figure()
    ax = plt.gca()
    
    alpha = 0.8,
    dark = 0.2,
    light = 0.85,
    rot = 0,
    intrv_cmap = sns.cubehelix_palette(start=-.3, rot=rot, dark=dark, light=light, reverse=True, as_cmap=True)
    color = None
    if hue is None:
        color = intrv_cmap(0.7)
    #intrv_cmap = sns.dark_palette("blue", as_cmap=True)
    sns.scatterplot(x="pc0", y="pc1", color=color, alpha=alpha, data=intrv_df, ax=ax, hue=hue, palette=intrv_cmap, edgecolor="none")
    
    native_cmap = sns.cubehelix_palette(start=0.7, rot=rot, dark=dark, light=light, reverse=True, as_cmap=True)
    if hue is None:
        color = native_cmap(0.7)
    #native_cmap = sns.dark_palette("red", as_cmap=True)
    sns.scatterplot(x="pc0", y="pc1", color=color, alpha=alpha, data=natty_df, ax=ax, hue=hue, palette=native_cmap, edgecolor="none")
                    #hue="hue", palette="blue")
        
    ## y divider
    #ax.plot([0,0],[-1,5], "k--", alpha=0.5)
    ## x dividers
    #for i in y_values[:-1]:
    #    y = i+0.5
    #    ax.plot([-2,2],[y,y], "k--", alpha=0.5)
    #plt.xlim([-2,2])
    #plt.ylim([-0.75,4.75])
    
    plt.xlabel("", fontsize=fontsize)
    plt.ylabel("", fontsize=fontsize)
    
    #plt.xticks([], fontsize=fontsize)
    #plt.yticks([], fontsize=fontsize)
    
    # # Manually create colorbars / legend patches
    # native_cmap = sns.cubehelix_palette(start=0.7, rot=rot, dark=dark, light=light, reverse=True, as_cmap=True)
    # intrv_cmap = sns.cubehelix_palette(start=-.3, rot=rot, dark=dark, light=light, reverse=True, as_cmap=True)
    
    # Legend handles: colored rectangles with labels
    native_patch = mpatches.Patch(color=native_cmap(0.8), label="Native")
    intrv_patch = mpatches.Patch(color=intrv_cmap(0.8), label="Intervened")
    
    if not incl_legend:
        plt.legend().set_visible(False)
    else:
        ax.legend(handles=[native_patch, intrv_patch], fontsize=legendsize, loc="upper right", bbox_to_anchor=(1.75,1))
    if save_name:
        plt.savefig(save_name, dpi=600, bbox_inches="tight")
        print("Saved to", save_name)
    
    plt.show()

In [ ]:
import os
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA
import pandas as pd

print("Natty PCA")
X = np.concatenate([
    natty_states.clone().detach().cpu().float().numpy(),
    intrv_states.clone().detach().cpu().float().numpy(),
], axis=0)
train_X = X
mu = train_X.mean(0)
sig = train_X.std(0)+1e-5
X = (X-mu)/(sig)
train_X = (train_X-mu)/(sig)
pca = PCA()
pca.fit(train_X)

vecs = pca.transform(X)
natty_vecs = torch.tensor(vecs[:len(natty_states)])
intrv_vecs = torch.tensor(vecs[len(natty_states):])

natty_df = {
    "pc0": natty_vecs[:,0],
    "pc1": natty_vecs[:,1],
    "hue": np.ones_like(natty_vecs[:,1]),
}
intrv_df = {
    "pc0": intrv_vecs[:,0],
    "pc1": intrv_vecs[:,1],
    "hue": np.ones_like(natty_vecs[:,1]),
}


interleave_scatter(
    natty_df,
    intrv_df,
    hue=None,
    title="", #f"Neural Space {varb}",
    incl_legend=False,
    save_name=os.path.join(
        "./", f"llama_boundless_pricetag.png",
    )
)



In [42]:
try:
    from geomloss import SamplesLoss
except ModuleNotFoundError:
    !pip install geomloss
    from geomloss import SamplesLoss

kwargs = { "loss": "sinkhorn", "p": 2, "blur": 0.05, }
loss_fn = SamplesLoss(**kwargs)
def get_emd(X,Y):
    return loss_fn(X,Y).item()

perm = torch.randperm(len(natty_vecs)).long()
div = get_emd(natty_vecs, intrv_vecs)
baseline = get_emd(natty_vecs[perm[:len(natty_vecs)//2]], natty_vecs[perm[len(natty_vecs)//2:]]) 
div_fraction = (div-baseline)/baseline
print("Divergence:", div)
print("Baseline:", baseline)
print("% Divergence:", div_fraction)